<center><h1>Module 8 - Word vectors and word embeddings</h1></center>

First, I want to give a shoutout to Allison Parrish (http://www.decontextualize.com/). She mixes machine learning with poetry (!) I really like her work and explanations. I borrowed her color mixing example.

In module 6, we built a co-occurence matrix that was 25K by 25K. I'd like to use something like it and try out KNN. But it is crazy to use a collection of 25K vectors as our crowd. I'd like something much smaller. That is what we will tackle in this module, reducing the comat to a reasonable size, i.e., 300.

But before jumping right to that, I'd like to do some build-up. There will be a bit of review here given we looked at the comat idea earlier. But I think it will give you some new insight. And will lead us toward what is called word-embeddings or word-vectors.

##Might also turn on GPU

Runtime then Change runtime type to GPU. Will use it later. This will reset your kernel so good to do it up front here.

##Before we get going

I moved this up front so it will hang in beginning and once I set it up, I can do a Runtime/run-after and not get hung.

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


## Animal similarity

We'll begin by considering a small subset of English: words for animals. Our task will be similar to module 6:  find similarities among these words and the creatures they designate. So instead of similarity between `monster` and `frankenstein` we will look for similarities between `kitten` and `hamster`. I am not going to build a co-occurence matrix. Instead, I will come up with 2 features that I think are important for animals. See below:

![Animal spreadsheet](http://static.decontextualize.com/snaps/animal-spreadsheet.png)

This spreadsheet associates a handful of animals with two numbers: their cuteness and their size, both in a range from zero to one hundred. The values themselves are simply based on personal judgment. Your taste in cuteness and evaluation of size may differ significantly from mine. As with all data, these data are simply a mirror reflection of the person who collected them. In other words, always have to be aware of human bias in problems like this.

Note that this is **not** a comat. Instead, it is a step toward what is called a word-vector. For instance the word `kitten` has a 2d vector 95,15.

Let's build the pandas version of the table shown.


In [0]:
animal_dict = {
    'kitten': [95,15],
    'hamster': [80,8],
    'tarantula': [8,3],
    'puppy': [90, 20],
    'crocodile': [5, 40],
    'dolphin': [60,45],
    'panda bear': [75, 40],
    'lobster': [2, 15],
    'capybara': [70, 30],
    'elephant': [65, 90],
    'mosquito': [1, 1],
    'goldfish': [25, 2],
    'horse': [50, 50],
    'chicken': [25, 15]
}

In [3]:
import pandas as pd
animal_table = pd.DataFrame.from_dict(animal_dict, orient='index', columns=['cuteness', 'size'])
animal_table

,cuteness,size
kitten,95,15
hamster,80,8
tarantula,8,3
puppy,90,20
crocodile,5,40
dolphin,60,45
panda bear,75,40
lobster,2,15
capybara,70,30
elephant,65,90


##New type of index

Notice that instead of having row indices of 0,1, etc, the indices are now words. We can still get the 0th row using iloc as shown below.

In [0]:
row0 = animal_table.iloc[0]  #gets first row no matter what the index

In [5]:
row0

cuteness    95
size        15
Name: kitten, dtype: int64

In [6]:
type(row0)

pandas.core.series.Series

In [7]:
row0.name  #a bit weird - series object has a name field taken from index

'kitten'

In [8]:
animal_table.columns  #but name is not in table

Index(['cuteness', 'size'], dtype='object')

Below will fail if you uncomment it and run it. Why? The loc method assumes you give it column names. So the 0 should be in the column index. This is confusing because pandas hides this column name. It just shows it on the left. I see `kitten` in the index but no 0.

In [0]:
#animal_table.loc[0, 'size']  #looks for index 0 and does not see it

In [10]:
animal_table.index

Index(['kitten', 'hamster', 'tarantula', 'puppy', 'crocodile', 'dolphin',
       'panda bear', 'lobster', 'capybara', 'elephant', 'mosquito', 'goldfish',
       'horse', 'chicken'],
      dtype='object')

In [11]:
animal_table.loc['kitten']  #finds row with index kitten

cuteness    95
size        15
Name: kitten, dtype: int64

In [12]:
animal_table.loc['kitten', 'size']  #finds row with index kitten and pulls out size value

15

Personally, I find the way pandas handles dataframes and series object a bit confusing. We have to live with it.


The values in the table give us a way to make determinations about which animals are similar. For example, try to answer the following question: Which animal is most similar to a `capybara`? You could go through the values one by one and use euclidean distance (or cosine similarity) to make that evaluation. This is quite similar to what we were doing with `ordered_distances`.

Let's try visualizing the data as points in 2-dimensional space:

![Animal space](http://static.decontextualize.com/snaps/animal-space.png)


##Bring in your library now

In [13]:
#your code
!rm -r  'w20_ds_library'
clone_url = f'https://github.com/FutureDeus/w20_ds_library.git'
!git clone $clone_url
from w20_ds_library import *

rm: cannot remove 'w20_ds_library': No such file or directory
Cloning into 'w20_ds_library'...
remote: Enumerating objects: 102, done.
remote: Counting objects: 100% (102/102), done.
remote: Compressing objects: 100% (101/101), done.
remote: Total 102 (delta 64), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (102/102), 23.62 KiB | 5.91 MiB/s, done.
Resolving deltas: 100% (64/64), done.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


Ok, let's check out the panda capybara distance. I am using euclidean distance because I think it is easier to visualize on the graph.

In [14]:
euclidean_distance([70, 30], [75, 40]) # panda and capybara  11.180339887498949

11.180339887498949

It is less than the distance between "tarantula" and "elephant":

In [15]:
euclidean_distance([8, 3], [65, 90]) # tarantula and elephant  104.0096149401583

104.0096149401583

Modeling animals in this way has a few other interesting properties. For example, you can pick an arbitrary point in "animal space" and then find the animal closest to that point. If you imagine an animal of size 25 and cuteness 30, you can easily look at the space to find the animal that most closely fits that description: the chicken.



I am going to write a special function to work with the animal table. I could probably rework my existing `ordered_distances` function, but decided easier to write this new one.

In [0]:
def ordered_animals(target_vector, table):
  #distance_list = [(row.name,euclidean_distance(target_vector, row.tolist())) for i,row in table.iterrows()]  #for list comprehension fans
  distance_list = []
  for i,row in table.iterrows():
    a_vector = row.tolist()
    d = euclidean_distance(target_vector, a_vector)
    distance_list.append((row.name,d))
  return sorted(distance_list, key=lambda pair: pair[1], reverse=False)

In [17]:
pup = animal_table.loc['puppy'].tolist()

ordered_animals(pup, animal_table)

[('puppy', 0.0),
 ('kitten', 7.0710678118654755),
 ('hamster', 15.620499351813308),
 ('capybara', 22.360679774997898),
 ('panda bear', 25.0),
 ('dolphin', 39.05124837953327),
 ('horse', 50.0),
 ('chicken', 65.19202405202648),
 ('goldfish', 67.446274915669),
 ('elephant', 74.33034373659252),
 ('tarantula', 83.74365647617735),
 ('crocodile', 87.32124598286491),
 ('lobster', 88.14193099768123),
 ('mosquito', 91.00549433962765)]

Let's look at it  geometrically. You can  answer questions like: what's halfway between a chicken and an elephant? Simply draw a line from "elephant" to "chicken," mark off the midpoint and find the closest animal. (According to our chart, halfway between an elephant and a chicken is a horse.) Let's check that out computationally.



In [18]:
elephant = animal_table.loc['elephant'].tolist()
chicken = animal_table.loc['chicken'].tolist()
half_vector = [(e+c)/2  for e,c in zip(elephant, chicken)]
half_vector  #[45.0, 52.5]

[45.0, 52.5]

In [19]:
ordered_animals(half_vector, animal_table)

[('horse', 5.5901699437494745),
 ('dolphin', 16.77050983124842),
 ('panda bear', 32.5),
 ('capybara', 33.63406011768428),
 ('crocodile', 41.907636535600524),
 ('elephant', 42.5),
 ('chicken', 42.5),
 ('goldfish', 54.31620384378864),
 ('puppy', 55.509008277936296),
 ('hamster', 56.61492736019362),
 ('lobster', 57.05479822065801),
 ('tarantula', 61.80008090609591),
 ('kitten', 62.5),
 ('mosquito', 67.73662229547618)]

You can also ask: what's the *difference* between a hamster and a tarantula? According to our plot, it's about seventy five units of cute (and a few units of size).

The relationship of "difference" is an interesting one, because it allows us to reason about *analogous* relationships. In the chart below, I've drawn an arrow from "tarantula" to "hamster" (in blue):

![Animal analogy](http://static.decontextualize.com/snaps/animal-space-analogy.png)

You can understand this arrow as being the *relationship* between a tarantula and a hamster, in terms of their size and cuteness (i.e., hamsters and tarantulas are about the same size, but hamsters are much cuter). In the same diagram, I've also transposed this same arrow (this time in red) so that its origin point is "chicken." The arrow ends closest to "kitten." What we've discovered is that the animal that is about the same size as a chicken but much cuter is... a kitten. To put it in terms of an analogy:

    Tarantulas are to hamsters as chickens are to kittens.
    


In [20]:
tarantula = animal_table.loc['tarantula'].tolist()
hamster = animal_table.loc['hamster'].tolist()
thd = euclidean_distance(tarantula, hamster)  #tarantula hamster diff
thd  #72.17340230306452

72.17340230306452

Now get animal distances from chicken and find the one closest to 72.17.

In [21]:
chick_ds = ordered_animals(chicken, animal_table)
chick_ds

[('chicken', 0.0),
 ('goldfish', 13.0),
 ('tarantula', 20.808652046684813),
 ('lobster', 23.0),
 ('mosquito', 27.784887978899608),
 ('crocodile', 32.01562118716424),
 ('horse', 43.01162633521314),
 ('dolphin', 46.09772228646444),
 ('capybara', 47.43416490252569),
 ('hamster', 55.44366510251645),
 ('panda bear', 55.90169943749474),
 ('puppy', 65.19202405202648),
 ('kitten', 70.0),
 ('elephant', 85.0)]

In [22]:
min(chick_ds, key=lambda x:abs(x[1]-thd))  #('kitten', 70.0)

('kitten', 70.0)


A set of vectors that are all part of the same data set is often called a *vector space*, e.g., the crowd table in KNN. The vector space of animals in this section has two *dimensions*, by which I mean that each vector in the space has two numbers associated with it (i.e., two columns in the spreadsheet). The fact that this space has two dimensions just happens to make it easy to *visualize* the space by drawing a 2D plot. But most vector spaces you'll work with will have more than two dimensions—sometimes many hundreds. In those cases, it's more difficult to visualize the "space," but the math works pretty much the same.

## Language with vectors: colors

So far, so good. We have a system in place—albeit highly subjective—for talking about animals and the words used to name them. I want to talk about another vector space that has to do with language: the vector space of colors.

Colors are often represented in computers as vectors with three dimensions: red, green, and blue. Just as with the animals in the previous section, we can use these vectors to answer questions like: which colors are similar? What's the most likely color name for an arbitrarily chosen set of values for red, green and blue? Given the names of two colors, what's the name of those colors' "average"?

We'll be working with this [color data](https://github.com/dariusk/corpora/blob/master/data/colors/xkcd.json) from the [xkcd color survey](https://blog.xkcd.com/2010/05/03/color-survey-results/). The data relates a color name to the RGB value associated with that color. [Here's a page that shows what the colors look like](https://xkcd.com/color/rgb/). Read the color data now.

In [0]:
import json


Here is link to json file. You will have to upload it to Google Drive and then read it: https://drive.google.com/open?id=1Zo35tEkTN9Z_G-YIPN7u6LaoXYEHqio-. So replace my path with your own.

In [0]:
color_data = json.loads(open("/content/gdrive/My Drive/class_tables/xkcd.json").read())

The following function converts colors from hex format (`#1a2b3c`) to a tuple of integers:

In [0]:
def hex_to_int(s:str) -> tuple:
  assert isinstance(s, str)
  assert s[0] == '#'
  assert len(s) == 7

  s = s.lstrip("#")
  return int(s[:2], 16), int(s[2:4], 16), int(s[4:6], 16)

In [26]:
hex_to_int('#1a2b3c')  #(26, 43, 60)

(26, 43, 60)

And the following cell creates a dictionary and populates it with mappings from color names to RGB vectors for each color in the data:

In [0]:
colors = {item["color"]: hex_to_int(item["hex"]) for item in color_data['colors']}

Testing it out:

In [28]:
colors['olive']  #(110, 117, 14)

(110, 117, 14)

In [29]:
colors['red']  #(229, 0, 0)

(229, 0, 0)

In [30]:
colors['black']  #(0, 0, 0)

(0, 0, 0)

#Challenge 1

We will need some functions for performing basic vector "arithmetic." These functions will work with vectors in spaces of any number of dimensions.

Here is where you finally get to use `numpy`. Please take my Python code and find the `numpy` equivalent of doing things.

Except for `fast_cosine`, all of my numpy functions were 1 line long. If you are getting longer, search a little harder in the numpy documentation.

First I will add some things to do type hints.

In [0]:
import numpy
from typing import TypeVar, Callable
narray = TypeVar('numpy.ndarray')


##Numpy equivalent of this please

In [0]:
a = [10,1]
b = [5,2]

In [33]:
euclidean_distance(a,b)  #5.0990195135927845

5.0990195135927845

In [0]:
import numpy as np  #standard way to do numpy import

In [0]:
def fast_euclidean_distance(x:narray, y:narray) -> float:
  assert isinstance(x, numpy.ndarray), f"x must be a numpy array but instead is {type(x)}"
  assert len(x.shape) == 1, f"x must be a 1d array but instead is {len(x.shape)}d"
  assert isinstance(y, numpy.ndarray), f"y must be a numpy array but instead is {type(y)}"
  assert len(y.shape) == 1, f"y must be a 1d array but instead is {len(y.shape)}d"
  
  return np.linalg.norm(x-y)


In [0]:
na = np.array(a)
nb = np.array(b)

In [37]:
fast_euclidean_distance(na,nb)  #5.0990195135927845


5.0990195135927845

##Numpy equivalent of this please

In [38]:
[c1 - c2 for c1, c2 in zip(a, b)]  #subtract one vector from another


[5, -1]

In [0]:
def subtractv(x:narray, y:narray) -> narray:
  assert isinstance(x, numpy.ndarray), f"x must be a numpy array but instead is {type(x)}"
  assert len(x.shape) == 1, f"x must be a 1d array but instead is {len(x.shape)}d"
  assert isinstance(y, numpy.ndarray), f"y must be a numpy array but instead is {type(y)}"
  assert len(y.shape) == 1, f"y must be a 1d array but instead is {len(y.shape)}d"

  return np.subtract(x, y)


In [40]:
subtractv(na,nb)  #array([ 5, -1])

array([ 5, -1])

##Numpy equivalent of this please

In [41]:
[c1 + c2 for c1, c2 in zip(a, b)]  #add one vector to another


[15, 3]

In [0]:
def addv(x:narray, y:narray) -> narray:
  assert isinstance(x, numpy.ndarray), f"x must be a numpy array but instead is {type(x)}"
  assert len(x.shape) == 1, f"x must be a 1d array but instead is {len(x.shape)}d"
  assert isinstance(y, numpy.ndarray), f"y must be a numpy array but instead is {type(y)}"
  assert len(y.shape) == 1, f"y must be a 1d array but instead is {len(y.shape)}d"
  
  return np.add(x, y)

In [43]:
addv(na,nb)  #array([15,  3])

array([15,  3])

##Numpy equivalent of this please

My `meanv_slow` function takes a list of vectors and finds their mean or average. Provide numpy equivalent.

In [0]:
def meanv_slow(coords):
    # assumes every item in coords has same length as item 0
    sumv = [0] * len(coords[0])
    for item in coords:
        for i in range(len(item)):
            sumv[i] += item[i]
    mean = [0] * len(sumv)
    for i in range(len(sumv)):
        mean[i] = float(sumv[i]) / len(coords)
    return mean


In [0]:
test = [[0, 1], [2, 2], [4, 3]]

In [46]:
meanv_slow(test)  #[2.0, 2.0]

[2.0, 2.0]

In [0]:
def meanv(matrix: narray) -> narray:
  assert isinstance(matrix, numpy.ndarray), f"matrix must be a numpy array but instead is {type(matrix)}"
  assert len(matrix.shape) == 2, f"matrix must be a 2d array but instead is {len(matrix.shape)}d"

  return np.mean(matrix, axis = 0)


In [48]:
ntest = np.array(test)
ntest

array([[0, 1],
       [2, 2],
       [4, 3]])

In [49]:
meanv(ntest)  #array([2., 2.])

array([2., 2.])

##Numpy equivalent of this please

You have a cosine_similarity function now. Create a new version called fast_cosine using numpy. Be careful to look out for 0-divide in the denominator!

In [0]:
from numpy.linalg import norm  #hint: i found this useful

def fast_cosine(v1:narray, v2:narray) -> float:
  assert isinstance(v1, numpy.ndarray), f"v1 must be a numpy array but instead is {type(v1)}"
  assert len(v1.shape) == 1, f"v1 must be a 1d array but instead is {len(v1.shape)}d"
  assert isinstance(v2, numpy.ndarray), f"v2 must be a numpy array but instead is {type(v2)}"
  assert len(v2.shape) == 1, f"v2 must be a 1d array but instead is {len(v2.shape)}d"
  assert len(v1) == len(v2), f'v1 and v2 must have same length but instead have {len(v1)} and {len(v2)}'

  bottom = (norm(v1)*norm(v2))
  return 0.0 if bottom == 0 else np.dot(v1, v2)/bottom


In [51]:
fast_cosine(np.array(colors['olive']), np.array(colors['red']))  #0.6823879113063314

0.6823879113063314

In [52]:
fast_cosine(np.array(colors['black']), np.array(colors['red']))  #0.0

0.0

#Challenge 2

Let's make life easier for ourselves. Convert colors and animal_dict to numpy versions and then define a new version of ordered_distances.

In [0]:
#your code here

n_colors = {keys:np.array(values) for keys, values in colors.items()}

In [54]:
n_colors['olive']  #array([110, 117,  14])

array([110, 117,  14])

In [55]:
type(n_colors['olive'])  #numpy.ndarray

numpy.ndarray

In [0]:
#your code here

n_animal_dict = {keys:np.array(values) for keys, values in animal_dict.items()}

In [57]:
n_animal_dict['chicken']  #array([25, 15])

array([25, 15])

In [58]:
type(n_animal_dict['chicken'])  #numpy.ndarray

numpy.ndarray

Just as a test, use numpy to show that the distance from "red" to "green" is greater than the distance from "red" to "pink".

In [59]:
fast_euclidean_distance(n_colors['red'],n_colors['green']) > fast_euclidean_distance(n_colors['red'],n_colors['pink'])  #True

True

### Finding the closest item

Just as we wanted to find the animal that most closely matched an arbitrary point in cuteness/size space, we'll want to find the closest color name to an arbitrary point in RGB space. The easiest way to find the closest item to an arbitrary vector is simply to find the distance between the target vector and each item in the space, in turn, then sort the list from closest to farthest. 

Note we want distances so use your fast_euclidean_distance function.

In [0]:
def dict_ordered_distances(space:dict, coord:narray) -> list:
  assert isinstance(space, dict), f"space must be a dictionary but instead a {type(space)}"
  assert isinstance(list(space.values())[0], numpy.ndarray), f"space must have numpy arrays as values but instead has {type(space.values()[0])}"
  assert isinstance(coord, numpy.ndarray), f"coord must be a numpy array but instead is {type(cord)}"
  assert len(list(space.values())[0]) == len(coord), f"space values must be same length as coord"
  assert len(coord) == 3, "coord must be a triple"

  #your code here
  return sorted([(keys, fast_euclidean_distance(values, coord)) for keys, values in space.items()], key = lambda x: x[1])

Testing it out, we can find the ten colors closest to "red":

In [61]:
dict_ordered_distances(n_colors, n_colors['red'])[:10]

[('red', 0.0),
 ('fire engine red', 25.079872407968907),
 ('bright red', 29.068883707497267),
 ('tomato red', 45.552167895721496),
 ('cherry red', 45.73838650411709),
 ('scarlet', 46.33573135281238),
 ('vermillion', 53.563046963368315),
 ('orangish red', 56.2672195865408),
 ('cherry', 56.49778756730214),
 ('lipstick red', 59.84981202978001)]

<pre>
[('red', 0.0),
 ('fire engine red', 25.079872407968907),
 ('bright red', 29.068883707497267),
 ('tomato red', 45.552167895721496),
 ('cherry red', 45.73838650411709),
 ('scarlet', 46.33573135281238),
 ('vermillion', 53.563046963368315),
 ('orangish red', 56.2672195865408),
 ('cherry', 56.49778756730214),
 ('lipstick red', 59.84981202978001)]
 </pre>

... or the ten colors closest to (150, 60, 150):

In [62]:
dict_ordered_distances(n_colors, np.array([150, 60, 150]))[:10]

[('warm purple', 15.684387141358123),
 ('medium purple', 16.0312195418814),
 ('ugly purple', 18.2208671582886),
 ('light eggplant', 23.2163735324878),
 ('purpleish', 27.586228448267445),
 ('purplish', 27.92848008753788),
 ('purply', 28.231188426986208),
 ('light plum', 33.74907406137241),
 ('purple', 38.88444419044716),
 ('muted purple', 40.8656334834051)]

<pre>
[('warm purple', 15.684387141358123),
 ('medium purple', 16.0312195418814),
 ('ugly purple', 18.2208671582886),
 ('light eggplant', 23.2163735324878),
 ('purpleish', 27.586228448267445),
 ('purplish', 27.92848008753788),
 ('purply', 28.231188426986208),
 ('light plum', 33.74907406137241),
 ('purple', 38.88444419044716),
 ('muted purple', 40.8656334834051)]
 </pre>

### Color magic

The magical part of representing words as vectors is that the vector operations we defined earlier appear to operate on language the same way they operate on numbers. For example, if we find the word closest to the vector resulting from subtracting "red" from "purple," we get a series of "blue" colors:

In [63]:
dict_ordered_distances(n_colors, subtractv(n_colors['purple'], n_colors['red']))[:10]

[('cobalt blue', 108.42970072816765),
 ('royal blue', 111.96428001822724),
 ('darkish blue', 112.76967677527502),
 ('true blue', 115.52056094046635),
 ('royal', 115.56383517346592),
 ('prussian blue', 116.18519699169941),
 ('dark royal blue', 116.73045875006231),
 ('deep blue', 117.957619508025),
 ('marine blue', 118.28778466096996),
 ('deep sea blue', 120.03332870498926)]

This matches our intuition about RGB colors, which is that purple is a combination of red and blue. Take away the red, and blue is all you have left.

You can do something similar with addition. What's blue plus green?

In [64]:
dict_ordered_distances(n_colors, addv(n_colors['blue'], n_colors['green']))[:10]

[('bright turquoise', 14.212670403551895),
 ('bright light blue', 15.0996688705415),
 ('bright aqua', 20.73644135332772),
 ('cyan', 27.49545416973504),
 ('neon blue', 33.34666400106613),
 ('aqua blue', 38.3275357934736),
 ('bright cyan', 42.49705872175156),
 ('bright sky blue', 45.05552130427524),
 ('aqua', 49.09175083453431),
 ('bright teal', 56.2672195865408)]

That's right, it's something like turquoise or cyan!


#Challenge 3

 What if we find the average of black and white? Predictably, we get gray:

 <pre>
 [u'medium grey',
 u'purple grey',
 u'steel grey',
 u'battleship grey',
 u'grey purple',
 u'purplish grey',
 u'greyish purple',
 u'steel',
 u'warm grey',
 u'green grey']
 </pre>

Let's make sure my answer is correct. First, build your matrix with the black row and the white row.

In [0]:
#your code below

bw_matrix = np.array([n_colors['black'], n_colors['white']])


In [66]:
bw_matrix  #array([[  0,   0,   0], [255, 255, 255]])

array([[  0,   0,   0],
       [255, 255, 255]])

<pre>
array([[  0,   0,   0],
       [255, 255, 255]])
</pre>

Now take the mean and find the closest colors to that mean.

In [0]:
#your code here

bw_list = dict_ordered_distances(n_colors, meanv(bw_matrix))[:10]

In [68]:
bw_list  #make sure matches list above

[('medium grey', 4.330127018922194),
 ('purple grey', 18.567444627627143),
 ('steel grey', 19.716744153130353),
 ('battleship grey', 21.511624764298954),
 ('grey purple', 22.46664193866097),
 ('purplish grey', 24.14021540914662),
 ('greyish purple', 24.264171117101856),
 ('steel', 25.470571253900058),
 ('warm grey', 26.12948526090784),
 ('green grey', 26.205915362757317)]

Just as with the tarantula/hamster example from the previous section, we can use color vectors to reason about relationships between colors. In the cell below, finding the difference between "pink" and "red" then adding it to "blue" seems to give us a list of colors that are to blue what pink is to red (i.e., a slightly lighter, less saturated shade):

In [69]:
# an analogy: pink is to red as X is to blue
pink_to_red = subtractv(n_colors['pink'], n_colors['red'])
dict_ordered_distances(n_colors, addv(pink_to_red, n_colors['blue']))[:10]

[('neon blue', 163.29727493133498),
 ('bright sky blue', 163.44418007380992),
 ('bright light blue', 170.0764533967004),
 ('cyan', 172.97976760303501),
 ('bright cyan', 174.54512310574592),
 ('bright turquoise', 176.39727889057698),
 ('clear blue', 178.23860412379804),
 ('azure', 178.54131174604942),
 ('dodger blue', 178.92456511055155),
 ('lightish blue', 180.95303258028034)]

Another example of color analogies: Navy is to blue as true green/dark grass green is to green:

In [70]:
navy_to_blue = subtractv(n_colors['navy'], n_colors['blue'])
dict_ordered_distances(n_colors, addv(navy_to_blue, n_colors['green']))[:10]

[('true green', 140.59160714637272),
 ('dark grass green', 143.85409274678284),
 ('grassy green', 147.770091696527),
 ('racing green', 148.82540105774956),
 ('forest', 151.07944929738127),
 ('bottle green', 151.52887513606112),
 ('dark olive green', 153.4079528577316),
 ('darkgreen', 153.6522046701576),
 ('forrest green', 154.042202009709),
 ('grass green', 154.52184311611094)]

The examples above are fairly simple from a mathematical perspective but nevertheless *feel* magical: they're demonstrating that it's possible to use math to reason about how people use language.

### Doing bad digital humanities with color vectors

With the tools above in hand, we can start using our vectorized knowledge of language toward academic ends. In the following example, I'm going to calculate the average color of Bram Stoker's *Dracula*.

I downloaded the text file from Project Gutenberg (http://www.gutenberg.org/cache/epub/345/pg345.txt) and placed it in my google drive. I'll give you the link in a minute.

First, we'll load [spaCy](https://spacy.io/). This is a package with many of the same features of nltk. You can read about the differences here: https://medium.com/@akankshamalhotra24/introduction-to-libraries-of-nlp-in-python-nltk-vs-spacy-42d7b2f128f2. The main thing we want from spacy are word-vectors, which are not available in nltk.

In [0]:
import spacy


Spacy is set up to use a GPU if available. You can turn on GPU in Runtime menu, Change runtime type. 

##However, bug for me

When I uncomment the line below, I get an error when calling the nlp parser. I did a little digging but did not find a fix. If someone finds a way to uncomment this and avoid later errors, let me know.

In [0]:
#spacy.prefer_gpu()  #True if have GPU turned on but bug comes later


Now pull in a vocabulary. You have choices here. But have to be careful. If you load the small versions, you will not get the word-vectors we want. So I am using the medium (md) version. For other choices see https://spacy.io/models/en.

In [72]:
!python -m spacy download en_core_web_md


     |████████████████████████████████| 95.4MB 5.6MB/s 
  Created wheel for en-core-web-md: filename=en_core_web_md-2.1.0-cp36-none-any.whl size=97126236 sha256=cd964f29c3b20d3a98131be446bd9c2867b063b4b8331a84b6aac2d9d3acee62
  Stored in directory: /tmp/pip-ephem-wheel-cache-2f8zt6ro/wheels/c1/2c/5f/fd7f3ec336bf97b0809c86264d2831c5dfb00fc2e239d1bb01
Successfully built en-core-web-md
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_md')


In [0]:
import en_core_web_md
nlp = en_core_web_md.load()  #Gives us a way to parse text documents in one line of code. You will see in minute.

In [0]:
spnlp = TypeVar('spacy.lang.en.English')  #for type hints

Let's do a little exploration of what we just loaded.

In [75]:
nlp.vocab.length  #1.3M words

1340242

In [76]:
nlp.vocab.__contains__('marvelous')  #dunder method for checking if word in vocab

True

In [77]:
nlp.vocab.__contains__('askfds')

False

##But wait, there's more

Besides giving us a dictionary (vocab) of 1.3M words, spacy also gives us a parser which is more powerful than our current `get_clean_words`. What I want you to do is feed an entire book, as one long string, to the parser. Then we can mess around with what is produced.

To calculate the average color of the entire Dracula book, we'll follow these steps:

1. Parse the book text into words using spacy's `nlp` method (actually class constructor).
2. Check every word to see if it names a color in our vector space, i.e., the n_colors dictionary. If it does, add it to a list of vectors.
3. Find the average of that list of vectors.
4. Find the color(s) closest to that average vector.

The following cell performs step 1. Here is link to the txt file: https://drive.google.com/open?id=1448pTxyIcgiI94wMyB6m4bIT_nZsmFHx. You will need to add to your own google drive folder to open and read it.

##Extra credit

I spent a little bit of time trying to find a way to directly use the url to read the file from google drive. I failed. The url is to a web page and I was getting the raw html. If someone finds a way to use the url directly, let me know for extra credit.

In [0]:
#this can take a minute - we are parsing all of Dracula!

import io
f = io.open("/content/gdrive/My Drive/class_tables/pg345.txt", mode="r", encoding="utf-8-sig")  #if use utf-8 get \ufeff BOM

drac_doc = nlp(f.read())  #this is the parser doing its thing. Cool.

In [79]:
len(drac_doc)  #209124 tokens were parsed.

209124

##Behind the scenes

The nlp parser does a lot of work for us. It breaks the text into sentences and words. It tags a word with its part of speech. It even builds a parse tree and places the token within that tree. And it does Name-Entity Recognition (NER) on tokens. Bottomline: it does a lot! You can see some of it here: https://spacy.io/api/token#attributes.

The parser returns a data structure, which I have in `drac_doc`, that holds all of this information. I don't want to get hung up on all the strange and wondrous ways you can use `drac_doc`. So I'll just give you the bare bones. 

In [80]:
type(drac_doc)

spacy.tokens.doc.Doc

In [81]:
type(drac_doc[0])  #looks like it is a sequence of tokens

spacy.tokens.token.Token

In [82]:
type(drac_doc[0].text)  #and I can the word text from the token as a string

str

In [83]:
drac_doc[0].text.lower()  #might want to lower the text

'the'

In [84]:
drac_doc[0].lower_  #this does the same

'the'

##In summary

The spacy parser gives a sequence of tokens. Each token holds lots of useful information, only a bit of which we will use for now.

##Steps 2 and 3: find the average vector

General strategy: go through every token in drac_doc, lower it and check if in n_colors. If it is, add its RGB vector to your matrix. When done, get the average RGB vector. Remember to use your meanv function for speed.

In [85]:
%%time

# your code below

avg_color = meanv(np.array([n_colors[i.lower_] for i in drac_doc if i.lower_ in n_colors]))

CPU times: user 198 ms, sys: 866 µs, total: 199 ms
Wall time: 204 ms


In [86]:
avg_color  #array([147.44839068, 113.65371809, 100.13540511])

array([147.44839068, 113.65371809, 100.13540511])

Now, we'll pass the averaged color vector to ordering function, yielding a brown mush, which is kinda what you'd expect from adding a bunch of colors together willy-nilly.

In [87]:
dict_ordered_distances(n_colors, avg_color)[:10]

[('reddish grey', 13.519858753013214),
 ('brownish grey', 15.356247186381948),
 ('brownish', 16.350106463486874),
 ('brown grey', 19.826822637698537),
 ('mocha', 21.824003657449868),
 ('grey brown', 26.730012587581818),
 ('puce', 28.095953180857567),
 ('dull brown', 28.286050911198767),
 ('pinkish brown', 29.719493432987974),
 ('dark taupe', 31.643437130672552)]

<pre>
[u'reddish grey',
 u'brownish grey',
 u'brownish',
 u'brown grey',
 u'mocha',
 u'grey brown',
 u'puce',
 u'dull brown',
 u'pinkish brown',
 u'dark taupe']
 </pre>

On the other hand, here's what we get when we average the colors of Charlotte Perkins Gilman's classic *The Yellow Wallpaper*.  The result definitely reflects the content of the story, so maybe we're on to something here.

Here is link to the txt file: https://drive.google.com/open?id=1lxympMbAxw_2f8CI5KPzKIS_k-zS7cQM. You will need to add to your own google drive folder to open and read it.

In [0]:

f = io.open("/content/gdrive/My Drive/class_tables/pg1952.txt", mode="r", encoding="utf-8-sig")
yellow_doc = nlp(f.read())

In [89]:
len(yellow_doc)  #11649 tokens

11649

Do the same thihg: find the average color vector for the book.

In [90]:
%%time

#your code here

avg_color = meanv(np.array([n_colors[i.lower_] for i in yellow_doc if i.lower_ in n_colors]))

CPU times: user 14.8 ms, sys: 0 ns, total: 14.8 ms
Wall time: 16.3 ms


In [91]:
avg_color  #array([192.        , 185.26923077,  48.23076923])

array([192.        , 185.26923077,  48.23076923])

In [92]:
dict_ordered_distances(n_colors, avg_color)[:10]

[('pea', 32.867606937512456),
 ('puke yellow', 34.6139529618472),
 ('sick green', 35.25580652163731),
 ('vomit yellow', 37.701902232548726),
 ('booger', 39.06073635071867),
 ('olive yellow', 39.34720654280166),
 ('snot', 40.548768942125655),
 ('gross green', 41.77193998863593),
 ('dirty yellow', 42.05000193486195),
 ('mustard yellow', 42.420635957392086)]

<pre>
[u'pea',
 u'puke yellow',
 u'sick green',
 u'vomit yellow',
 u'booger',
 u'olive yellow',
 u'snot',
 u'gross green',
 u'dirty yellow',
 u'mustard yellow']
 </pre>

Definitely captures the yellowness (in kind of a gross way!).

#What have we learned?

One means of converting a sequence of words into a single vector is to take the average of all the individual word vectors. We are taking the entire set of words in a book and averaging. But we can also do the same thing with smaller units, e.g., sentences, tweets. We will use that idea shortly. But next I want to return to the comat idea.


## Distributional semantics

In the previous section, the examples are interesting because of a simple fact: colors that we think of as similar are "closer" to each other in RGB vector space. In our color vector space, or in our animal cuteness/size space, you can think of the words identified by vectors close to each other as being *synonyms*, in a sense: they sort of "mean" the same thing.  Think of this in terms of writing, say, a search engine. If someone searches for "mauve trousers," then it's probably also okay to show them results for, say,

In [93]:
for cname, dist in dict_ordered_distances(n_colors, n_colors['mauve'])[:10]:
    print(cname + " trousers")

mauve trousers
dusty rose trousers
dusky rose trousers
brownish pink trousers
old pink trousers
reddish grey trousers
dirty pink trousers
old rose trousers
light plum trousers
ugly pink trousers


That's all well and good for color words, which intuitively seem to exist in a multidimensional continuum of perception, and for our animal space, where we've written out the vectors ahead of time. But what about arbitrary words? Is it possible to create a vector space for all English words that has this same "closer in space is closer in meaning" property?

To answer that, we have to back up a bit and ask the question: what does *meaning* mean? No one really knows, but one theory popular among computational linguists, computer scientists and other people who make search engines is the [Distributional Hypothesis](https://en.wikipedia.org/wiki/Distributional_semantics), which states that:

    Linguistic items with similar distributions have similar meanings.
    
What's meant by "similar distributions" is *similar contexts*. Take for example the following sentences:

    It was really cold yesterday.
    It will be really warm today, though.
    It'll be really hot tomorrow!
    Will it be really cool Tuesday?
    
According to the Distributional Hypothesis, the words `cold`, `warm`, `hot` and `cool` must be related in some way (i.e., be close in meaning) because they occur in a similar context, i.e., between the word "really" and a word indicating a particular day. (Likewise, the words `yesterday`, `today`, `tomorrow` and `Tuesday` must be related, since they occur in the context of a word indicating a temperature.)

In other words, according to the Distributional Hypothesis, a word's meaning is just a big list of all the contexts it occurs in. Two words are closer in meaning if they share contexts.

## Word vectors by counting contexts

So how do we turn this insight from the Distributional Hypothesis into a system for creating general-purpose vectors that capture the meaning of words?  Let's use a small source text to begin with, such as this excerpt from Dickens:

    It was the best of times, it was the worst of times.

This spreadsheet tries to capture the context of words. It is very similar to the comat of module 6.

![dickens contexts](http://static.decontextualize.com/snaps/best-of-times.png)

The spreadsheet has one column for every possible context, and one row for every word. The values in each cell correspond with how many times the word occurs in the given context. The numbers in the columns constitute that word's vector, i.e., the vector for the word `of` is

    [0, 0, 0, 0, 1, 0, 0, 0, 1, 0]

As I said, similar to the comat you built in week 6.
    
Because there are ten possible contexts, this is a ten dimensional space. You could use the same distance formula that we defined earlier to get useful information about which vectors in this space are similar to each other. In particular, the vectors for `best` and `worst` are actually the same (a distance of zero), since they occur only in the same context (`the ___ of`):

    [0, 0, 0, 1, 0, 0, 0, 0, 0, 0]
    
Of course, the conventional way of thinking about "best" and "worst" is that they're *antonyms*, not *synonyms*. But they're also clearly two words of the same kind, with related meanings (through opposition), a fact that is captured by this distributional model.

### Contexts and dimensionality

In a corpus of any reasonable size, there will be many thousands if not many millions of possible contexts. We saw that in module 6. It turns out, though, that many of the dimensions end up being superfluous and can either be eliminated or combined with other dimensions without significantly affecting the predictive power of the resulting vectors. The process of getting rid of superfluous dimensions in a vector space is called [dimensionality reduction](https://en.wikipedia.org/wiki/Dimensionality_reduction), and most implementations of count-based word-vectors make use of dimensionality reduction so that the resulting vector space has a reasonable number of dimensions (say, 100—300, depending on the corpus and application).

The question of how to identify a "context" is itself very difficult to answer. In the toy example above, we've said that a "context" is just the word that precedes and the word that follows. Depending on your implementation of this procedure, though, you might want a context with a bigger "window" (e.g., two words before and after), or a non-contiguous window (skip a word before and after the given word). You might exclude certain "function" words like "the" and "of" when determining a word's context, or you might [lemmatize](https://en.wikipedia.org/wiki/Lemmatisation) the words before you begin your analysis, so two occurrences with different "forms" of the same word count as the same context. These are all questions open to research and debate, and different implementations of procedures for creating count-based word vectors make different decisions on this issue. In module 6, we eliminated stop words but we did not go as far as lemmatizing.

### GloVe vectors

But you don't have to create your own word vectors from scratch! Many researchers have made downloadable databases of pre-trained vectors. One such project is Stanford's [Global Vectors for Word Representation (GloVe)](https://nlp.stanford.edu/projects/glove/). These 300-dimensional vectors are included with spaCy, and they're the vectors we'll be using for the rest of this module. In fact, you already have them. They come with `en_core_web_md`. Nice.

Check this out.

In [94]:
nlp.vocab.has_vector('frankenstein')  #check to make sure word vectors have been loaded

True

In [0]:
dogv = nlp.vocab['dog'].vector  #get the 300d vector for dog

In [96]:
type(dogv)

numpy.ndarray

In [97]:
len(dogv)  #all spacy word vectors are length 300

300

You can also get a unique id for each word. This id will allow you to get a vector. So it is another way of getting the vector for a word.

In [98]:
dog_id = nlp.vocab.strings['dog']
dog_id  #7562983679033046312


7562983679033046312

In [0]:
dogv2 = nlp.vocab.vectors[dog_id]  #another way to get a word-vector given you have word id



In [100]:
all(dogv == dogv2)

True

For the sake of convenience, the following function gets the vector of a given string from spaCy's vocabulary:

In [0]:
def vec(nlp:spnlp, s:str) -> narray:
    return nlp.vocab[s].vector

In [102]:
all(vec(nlp,'dog') == dogv)

True

In [103]:
all(vec(nlp,'afskfsd') == 0.)  #True - a 0 vector for non-words

True

##Looks like a 0 vector for non-words

Good to know.

The following cell shows that the cosine similarity between `dog` and `puppy` is larger than the similarity between `trousers` and `octopus`, thereby demonstrating that the vectors are working how we expect them to:

In [104]:
fast_cosine(vec(nlp,'dog'), vec(nlp,'puppy')) > fast_cosine(vec(nlp,'trousers'), vec(nlp,'octopus'))

True

# Sentence similarity

I am going to switch gears a bit, and move us closer to doing prediction. What I will be interested in is converting an entire sentence into a single glove vector. We can practice with the gothic authors. Here is general idea. I'll go through each row and grab the text of the sentence. I will *not* preprocess it, at least for now. Instead I will feed it directly to spacy's nlp method.

I'll then get the vectors for all the words in the sentence (all the tokens) and take their average. I will use that resulting vector as the represnetation of the sentence.

And as always, when I say "I" I mean you.



#Challenge 4

Go ahead and fill out sent2vec below. s is the raw string version of a sentence.

Use nlp to parse into tokens then get vectors of those tokens. You can see that you will get punctuation with the parse.

In [0]:
a_doc = nlp('Now is the time.'.lower())

In [106]:
for w in a_doc:
  print(w)

now
is
the
time
.


It is interesting that spacy gives vectors to punctuation.

In [107]:
y = vec(nlp,'.')
len(y)  #normal 300 length vector

300

In [108]:
a_doc[4]

.

In [109]:
a_doc[4].is_punct  #you can check a token by this

True

You could filter out punctuation but go ahead and use all the tokens you get from a spacy parse for now, including punctuation.

In [0]:
def sent2vec(nlp:spnlp, s: str) -> narray:

  return meanv(np.array([i.vector for i in nlp(s)]))


Test it out.

In [0]:
s1vec = sent2vec(nlp, 'it was the best of times and the worst of times.')

In [112]:
print(s1vec[:20])  #[ -0.07223818  0.20577    -0.06873892 -0.28904667 -0.01813886  0.01893845 ... -0.167774   -0.10611192]

[-0.07223818  0.20577    -0.06873892 -0.28904667 -0.01813886  0.01893845
 -0.0079125   0.00298117  0.1157698   2.459283   -0.21819901  0.10764957
  0.1087811  -0.10047682 -0.11054987 -0.18445301 -0.12053668  1.127795
 -0.167774   -0.10611192]


Let's find the sentence in our text file that is closest in "meaning" to an arbitrary input sentence. First, we'll get the list of sentences:

In [0]:
sentences = list(drac_doc.sents)  #kind of cool. spacy keeps track of sentences for us in sents attribute

In [114]:
type(sentences)

list

In [115]:
sentences[1]

This eBook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever.  

##Important note

While it may look like sentences holds a list of strings, it does not. Instead it holds a list of spacy Spans.

In [116]:
type(sentences[1])

spacy.tokens.span.Span

The following function takes a list of sentences from a spaCy parse and compares them to an input sentence, sorting them by cosine similarity.

I'm going to give you the code to avoid you having to delve into the innards of spacy Spans. I did the delving for you :) But note that `[w.vector for w in x]` asks for each token w in span x and then grabs the vector of w.

In [0]:
def spacy_closest_sent(nlp:spnlp, space:list, input_str:str, n:int=10):
  assert isinstance(space, list)
  assert all([isinstance(sp, spacy.tokens.span.Span) for sp in space])

  input_vec = sent2vec(nlp, input_str)
  return sorted(space,
                key=lambda x: fast_cosine(np.mean([w.vector for w in x], axis=0), input_vec),
                reverse=True)[:n]

Here are the sentences in *Dracula* closest in meaning to "My favorite food is strawberry ice cream." (Extra linebreaks are present because we didn't strip them out when we originally read in the source text.)

In [118]:
for sent in spacy_closest_sent(nlp, sentences, "My favorite food is strawberry ice cream.", 10):
    print( sent.text)
    print( "---")

This, with some cheese
and a salad and a bottle of old Tokay, of which I had two glasses, was
my supper.
---
I set to and
enjoyed a hearty meal.
---
There was everywhere a bewildering mass of fruit blossom--apple,
plum, pear, cherry; and as we drove by I could see the green grass under

---
I had for breakfast more paprika, and a sort of porridge of maize flour
which they said was "mamaliga," and egg-plant stuffed with forcemeat, a
very excellent dish, which they call "impletata.
---
I had for dinner, or
rather supper, a chicken done up some way with red pepper, which was
very good but thirsty.
---
I dined on what they
called "robber steak"--bits of bacon, onion, and beef, seasoned with red
pepper, and strung on sticks and roasted over the fire, in the simple
style of the London cat's meat!
---
We get hot soup, or coffee, or tea; and
off we go.
---
I got a cup of tea at the Aërated Bread Company
and came down to Purfleet by the next train.


---
There is not even a toilet glass on my
t

#Congrats

You are done with part A. The important point now is to collect your functions into your library. You will need them in part B.

My strategy was to create a new file in my repository to hold the functions I defined in this notebook. It is only these that we will need going forward. So decided to separate them out. You can load just the single file as so:
<pre>
from steve_rep.scacy_fns import *
</pre>
where `spacy_fns.py` is the file I created under repository `steve_rep`. When you are done with that, move on to part B.